In [1]:
import pandas as pd
pd.options.display.float_format = "{:,.2f}".format

<p style="font-family: Open Sans; color: #2B6264; font-weight: bold; font-size: 125%">What are we going to cover?</p>

<ul>
    <li style="color: #ff5200"><p style="font-family: Open Sans; color: #2B6264">How can we view the entire history of a Property's value using the LUSID APIs via the Python SDK?</p></li>
</ul>

<ul>
    <li style="color: #ff5200"><p style="font-family: Open Sans; color: #2B6264">How can we use the Property history to understand what the value of the Property is at any point in bi-temporal space?</p></li>
</ul>

<p style="font-family: Open Sans; color: #2B6264; font-weight: bold; font-size: 125%">Retrieving the History of a Time-Variant Property's Values</p>

In [ ]:
import pprint
import os
import lusid
from lusidjam import RefreshingToken
from finbourne_sdk_utils.pandas_utils.lusid_pandas import lusid_response_to_data_frame
from lusid.extensions import (
    SyncApiClientFactory,
    ArgsConfigurationLoader,
    EnvironmentVariablesConfigurationLoader,
    SecretsFileConfigurationLoader
)
from lusid.models import *

# Authenticate to SDK
# Run the Notebook in Jupyterhub for your LUSID domain and authenticate automatically
secrets_path = os.getenv("FBN_SECRETS_PATH")
# Run the Notebook locally using a secrets file (see https://support.lusid.com/docs/how-do-i-use-an-api-access-token-with-the-lusid-sdk)
if secrets_path is None:
    secrets_path = os.path.join(os.path.dirname(os.getcwd()), "secrets.json")

# Initiate an API Factory which is the client side object for interacting with LUSID APIs
config_loaders=[
    ArgsConfigurationLoader(access_token = RefreshingToken(), app_name = "LusidJupyterNotebook"),
    EnvironmentVariablesConfigurationLoader(),
    SecretsFileConfigurationLoader(secrets_path)]
api_factory = SyncApiClientFactory(config_loaders=config_loaders)

instruments_api = api_factory.build(lusid.api.InstrumentsApi)
display(instruments_api)


In [ ]:
instruments_api.upsert_instruments(
    scope="FinbourneUniversity",
    request_body={
        "instr_1": lusid.InstrumentDefinition(
            name="MICROSOFT CORP",
            identifiers={
                "Figi": lusid.InstrumentIdValue(
                    value="BBG000BPH459"
                )
            }
        )
    }
)

In [4]:
property_definitions_api = api_factory.build(lusid.api.PropertyDefinitionsApi)

try:
    property_definitions_api.delete_property_definition(
        domain="Instrument",
        scope="FinbourneUniversity",
        code="MarketCapitalisation")
except:
    pass

In [5]:
def upsert_instrument_properties(value, effective_from=None, effective_until=None):
    
    if value is None:
        property_value = None
    else:
        property_value = lusid.PropertyValue(
            metric_value=lusid.MetricValue(
                value=value,
                unit="USD"
            )
        )

    print(instruments_api.upsert_instruments_properties(
        scope="FinbourneUniversity",
        upsert_instrument_property_request=[
            lusid.UpsertInstrumentPropertyRequest(
                identifier_type="Figi",
                identifier="BBG000BPH459",
                properties=[
                    lusid.ModelProperty(
                        key="Instrument/FinbourneUniversity/MarketCapitalisationv2",
                        value=property_value,
                        effective_from=effective_from,
                        effective_until=effective_until
                    )
                ]
            )
        ]
    ))

In [ ]:
property_definitions_api = api_factory.build(lusid.api.PropertyDefinitionsApi)

try:
    property_definitions_api.get_property_definition(
        domain="Instrument",
        scope="FinbourneUniversity",
        code="MarketCapitalisationv2")
except:
    issuer_prop_definition = property_definitions_api.create_property_definition(
        create_property_definition_request=lusid.CreatePropertyDefinitionRequest(
            domain="Instrument",
            scope="FinbourneUniversity",
            code="MarketCapitalisationv2",
            display_name="Market Capitalisation",
            property_description="The market capitalisation of the instrument",
            data_type_id=lusid.ResourceId(
                scope="system",
                code="currencyAndAmount"
            ),
            life_time="TimeVariant",
            constraint_style="Property"
        )
    )
    
    upsert_instrument_properties(780360000000)
    upsert_instrument_properties(1200000000000, effective_from="2020-01-01T00:00:00Z")
    upsert_instrument_properties(1680000000000, effective_from="2021-01-01T00:00:00Z", effective_until="2022-01-01T00:00:00Z")
    upsert_instrument_properties(None, effective_from=None, effective_until="2018-12-31T00:00:00Z")
    upsert_instrument_properties(None, effective_from="2022-01-01T00:00:00Z")

In [ ]:
result = instruments_api.get_instrument_property_time_series(
    identifier_type="Figi",
    identifier="BBG000BPH459",
    scope="FinbourneUniversity",
    property_key="Instrument/FinbourneUniversity/MarketCapitalisationv2")

result_df = lusid_response_to_data_frame(result)
result_df = result_df.sort_values(by="asAtRange.fromDate").reset_index(drop=True)
result_df


In [ ]:
result_df.loc[0].to_frame().transpose()

<img src="./data/History-1.png" width="850"/>

In [ ]:
result_df.loc[1].to_frame().transpose()

<img src="./data/History-2.png" width="850"/>

In [ ]:
result_df.loc[2].to_frame().transpose()

<img src="./data/History-3.png" width="850"/>

In [ ]:
result_df.loc[3].to_frame().transpose()

<img src="./data/History-4.png" width="850"/>

In [ ]:
result_df.loc[4].to_frame().transpose()

<img src="./data/History-5.png" width="850"/>

In [ ]:
result_df.loc[5].to_frame().transpose()

<img src="./data/History-6.png" width="850"/>

In [ ]:
result_df.loc[6].to_frame().transpose()

<img src="./data/History-7.png" width="850"/>

<img src="./data/History-Complete.png" width="1200"/>

<img src="./data/History-Complete-Comparison.png" width="1400"/>

<p style="font-family: Open Sans; color: #2B6264; font-weight: bold; font-size: 125%">What have we covered?</p>

<ul>
    <li style="color: #ff5200"><p style="font-family: Open Sans; color: #2B6264">We retrieved the time series history for our Instrument Property representing Market Capitalisation that we populated in the previous module.</p></li>
</ul>

<ul>
    <li style="color: #ff5200"><p style="font-family: Open Sans; color: #2B6264">We saw how the Property's values were described by a combination of "Effective" and "AsAt" ranges.</p></li>
</ul>

<ul>
    <li style="color: #ff5200"><p style="font-family: Open Sans; color: #2B6264">Using these ranges we were able to construct a graph which allowed us to understand the value of the Property at any point in bi-temporal time.</p></li>
</ul>

<ul>
    <li style="color: #ff5200"><p style="font-family: Open Sans; color: #2B6264">We saw how we can modify the "AsAt" date on the request to change how much of the Property's history to return.</p></li>
</ul>

<ul>
    <li style="color: #ff5200"><p style="font-family: Open Sans; color: #2B6264">We discussed how any value which exists at the provided "AsAt" time is "prevailing" while all other values are "superseded".</p></li>
</ul>